# Домашнее задание 2. Извлечение коллокаций + NER

1. (3 балла) Предложите 3 способа найти упоминания товаров в отзывах. **yes**

2. (2 балла) Реализуйте один из предложенных вами способов.

3. (1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

4. (3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика  ранжирует выше коллокации, подходящие для отчёта).

5. (1 балл) Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров.


Должны получиться примерно такие группы:
```
watch 
--- 
stylish watch
good watches
great watch
love this watch
...
```

**Бонус** (2 балла): 
если придумаете способ объединить синонимичные упоминания (например, "Samsung Galaxy Watch", "watch", "smartwatch")


## 1. Способы найти упоминания товаров в отзывах

### Способ 1: Собственный W2V + мини-словарь

1. На текстах всех отзывов обучаю свою модель W2V
2. Вручную собираю небольшой список интересующих меня товаров ```serum, parfume, cleanser```
3. Чтобы найти возможные упоминания сывороток смотрю на ближайших соседей ```serum```. Предполагаю, что они будут названиями продуктов, потому что должны были использоваться в тех же контекстах.

#### Достоинства
1. Очень гибкий способ получить названия, так он не привязан ни к каким шаблонам
2. Кроме названий можно сразу получить качества упоминаемых товаров

#### Недостатки
1. Легко найти одноловные упоминания товаров, а двух и трех словные сложнее
2. При маленьком корпусе результаты могут разочаровать

### Способ 2: Метаданные + Комбинаторика

1. Из метаданных беру все названия брендов ```Shiseido```
2. В названиях товаров отсекаю называние бренда и получаю сами названия ```Shiseido White Lucent Brightening Moisturizing Emulsion``` --> ```White Lucent Brightening Moisturizing Emulsion```
3. Из токенов названия получаю возможные названия ```White Lucent Brightening Moisturizing Emulsion```: ```Brightening Moisturizing Emulsion```, ```White Lucent Emulsion```, ```White Lucent```.

#### Достоинства
1. Получившиеся паттерны описывают конкретные товары
2. Не требует много данных, достаточно списка бренда и товаров.

#### Недостатки
1. Основано на предположении, что в отзыве на товар будет указано его настоящее название или хотя бы его часть
2. Плохо поддается генерализации


### Способ 3: Заголовок отзыва + Шаблоны

1. Беру все заголовки отзывов
2. Шаблонами ищу в них именные группы ```Wonderful Skin Care Product``` -> Adj Noun Noun Noun -> ```Skin Care Product```

#### Достоинства
1. Паттерны быстро пишутся

#### Недостатки
1. Может быть много лишних матчей
2. Привязка к морфлогочиескому таггеру, а их идеальных не бывает

## 2. Реализуйте один из предложенных вами способов

### Я буду использовать первый способ

Сначала преобразую [мои данные](http://snap.stanford.edu/data/amazon/Beauty.txt.gz) в один большой мета-документ, где нет пунктуации, цифр, стоп-слов, регистр нижний, каждое предложение на отдельной строке.

In [2]:
with open('Beauty.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [22]:
len(lines)

2772616

In [26]:
content_lines = []

for line in lines:
    if line.startswith('product/productId'):
        continue
    elif line.startswith('product/title'):
        continue
    elif line.startswith('product/price'):
        continue
    elif line.startswith('review/userId'):
        continue
    elif line.startswith('review/profileName'):
        continue
    elif line.startswith('review/helpfulness'):
        continue
    elif line.startswith('review/score'):
        continue
    elif line.startswith('review/time'):
        continue
    else:
        content_lines.append(line)

In [27]:
clean_content_lines = []
for line in content_lines:
    if line.startswith('review/summary:') or line.startswith('review/text:'):
        clean_content_lines.append(line.split(':')[1])
    else:
        clean_content_lines.append(line)

In [40]:
len(clean_content_lines)

756168

In [47]:
import spacy
nlp = spacy.load('en_core_web_sm')

/Users/elizavetaersova/miniconda3/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


#### ¡Мне пришлось прервать следующую ячейку на середине выполнения, потому что очень долго!

In [ ]:
%%time
all_sentences = []
for line in clean_content_lines:
    doc = nlp(line)
    sentences = [sent.string.strip() for sent in doc.sents]
    all_sentences.extend(sentences)

In [53]:
with open('sentences.txt', 'w+', encoding='utf-8') as f:
    for sent in all_sentences:
        f.write(sent)

In [51]:
import re
import pattern
from pattern.en import lemma

In [54]:
with open('extended_stopwords_towards_nlp.txt', 'r', encoding='utf-8') as file:
    raw = file.read()
stopwords = raw.split()

In [66]:
def preproc(text):
    line = text.lower()
    clean_text = re.sub("[^A-Za-z']", ' ', line)
    if len(clean_text) > 2:
        line_lemmas = [lemma(wd) for wd in clean_text.split() if lemma(wd) not in stopwords]
        lemmatized = " ".join(line_lemmas)
        if len(lemmatized) > 2:
            return lemmatized
    else:
        return None

In [67]:
%%time
preproc_sent = []
for sent in all_sentences:
    clean_lemmas = preproc(sent)
    if clean_lemmas is not None:
        preproc_sent.append(clean_lemmas)

CPU times: user 1min 36s, sys: 576 ms, total: 1min 37s
Wall time: 1min 38s


#### Вот так выглядит предобработанное сообщение

In [71]:
preproc_sent[42]

'hair pretty curly short will frizz crazy'

In [73]:
len(preproc_sent)

940984

In [75]:
with open('w2v_doc.txt', 'w+', encoding='utf-8') as f:
    for sent in preproc_sent[:100000]:
        f.write(f"{sent}\n")

#### Натренирую собственный w2v на почти 1 млн предложений

In [83]:
import re
import gensim
import pandas as pd
from gensim.models import word2vec

In [84]:
f = 'w2v_doc.txt'
data = gensim.models.word2vec.LineSentence(f)

In [85]:
%time model_beauty = gensim.models.Word2Vec(data, size=300, max_vocab_size = 100000, window=3, min_count=2, workers=2)

CPU times: user 1min 49s, sys: 1.32 s, total: 1min 51s
Wall time: 1min 2s


In [86]:
model_beauty.init_sims(replace=True)
model_path = "beauty.bin"

print("Saving model...")
model_beauty.wv.save_word2vec_format(model_path, binary=True)

Saving model...


In [87]:
len(model_beauty.wv.vocab)

27413

Составлю список продуктов, для которых хочу делать поиск упоминаний

In [88]:
product_list = ['serum', 'moisturizer', 'cleanser', 'oil', 'perfume', 'sunscreen']

Найду самые близкие к ним слова

In [90]:
possible_colloc = []
for product in product_list:
    closest = model_beauty.wv.most_similar(product, topn=10)
    possible_colloc
    print(f'Для {product} ближайшие соседи: {closest}')

Для serum ближайшие соседи: [('masque', 0.718429446220398), ('booster', 0.6737878322601318), ('recovery', 0.6674602031707764), ('creme', 0.6610598564147949), ('retinol', 0.657066822052002), ('defense', 0.6453754901885986), ('mask', 0.6380693912506104), ('hydration', 0.6338380575180054), ('intensive', 0.632432222366333), ('vitamin', 0.6315939426422119)]
Для moisturizer ближайшие соседи: [('moisturiser', 0.6949144005775452), ('cream', 0.6875280141830444), ('moisterizer', 0.6725209951400757), ('lotion', 0.6603338718414307), ('sunblock', 0.6538445949554443), ('moisiturizer', 0.6340392827987671), ('cleanser', 0.618161678314209), ('sunscreen', 0.6172863841056824), ('toner', 0.6126477718353271), ('spf', 0.609116792678833)]
Для cleanser ближайшие соседи: [('toner', 0.7895275950431824), ('cleanse', 0.7720011472702026), ('astringent', 0.7232605814933777), ('exfoliant', 0.7053558826446533), ('exfoliator', 0.690445601940155), ('cetaphil', 0.6559640169143677), ('scrub', 0.6545692682266235), ('exfol

## 3. Соберу n-граммы с полученными сущностями

Чтобы найти сочетания с ними запущу поиск триграмм, возьму из всех триграмм только те, где есть два нужных мне слова.

In [1]:
with open('w2v_doc.txt', 'r', encoding='utf-8') as f:
    tokens = f.read().split()

In [2]:
import nltk
from nltk.collocations import *

In [3]:
%%time
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(tokens)

CPU times: user 31.1 s, sys: 526 ms, total: 31.7 s
Wall time: 32.6 s


In [ ]:
finder

## 4. Ранжирую n-граммы с помощью 3 коллокационных метрик

### PMI

In [4]:
finder.nbest(trigram_measures.pmi, 10)

[("'boost", "moussee'", 'extrodinary'),
 ("'goods'", "'could", 'improved'),
 ("'hyaluronic", "acid'", 'purporte'),
 ("'metro'", "'ryan", "seacrest'"),
 ('aefjor', 'aroijmralkjierg', 'rgeeiojgpokgear'),
 ('aerosolize', 'airborne', 'assoc'),
 ('ajo', 'marisco', 'pescado'),
 ('amarelece', 'aconselho', 'vivamente'),
 ('amouage', 'lorenzo', 'villoresi'),
 ('aroijmralkjierg', 'rgeeiojgpokgear', 'esoijfgoij')]

### t-score

In [5]:
finder.nbest(trigram_measures.student_t, 10)

[('love', 'thi', 'product'),
 ('recommend', 'thi', 'product'),
 ('buy', 'thi', 'product'),
 ('thi', 'product', 'year'),
 ('thi', 'product', 'work'),
 ('highly', 'recommend', 'thi'),
 ('love', 'thi', 'stuff'),
 ('love', 'thi', 'perfume'),
 ('love', 'love', 'love'),
 ('purchase', 'thi', 'product')]

### chi square

In [6]:
finder.nbest(trigram_measures.chi_sq, 10)

[("'boost", "moussee'", 'extrodinary'),
 ("'goods'", "'could", 'improved'),
 ("'hyaluronic", "acid'", 'purporte'),
 ("'metro'", "'ryan", "seacrest'"),
 ('aefjor', 'aroijmralkjierg', 'rgeeiojgpokgear'),
 ('aerosolize', 'airborne', 'assoc'),
 ('ajo', 'marisco', 'pescado'),
 ('amarelece', 'aconselho', 'vivamente'),
 ('amouage', 'lorenzo', 'villoresi'),
 ('aroijmralkjierg', 'rgeeiojgpokgear', 'esoijfgoij')]

## 5. Группирую полученные коллокации по NE